## MAKE SURE you updated your ssh key and you have a stable internet connection

In [ ]:
import os
import re
import subprocess
from datetime import datetime
import yaml
import ipywidgets as ipw
from IPython.display import display

In [ ]:
new_host_label = 'daint.alps'
remotehost = "daint.alps.cscs.ch"  # Ensure this is the correct hostname
proxy = "ela.cscs.ch"
# labels for paths
alps_files = "/home/jovyan/aiidalab-alps-files"
config_path = f"/home/jovyan/.ssh/config"
config_source = f"/home/jovyan/aiidalab-alps-files/config"
config_without_ela = f"/home/jovian/aiidalab-alps-files/config_without_ela"

# labels to rename old host and codes
relabeled_host = datetime.now().strftime("Used_till_%Y%m%d%H%M")+'_'+new_host_label
relabeled_code = datetime.now().strftime("Used_till_%Y%m%d%H%M")+'_'

In [ ]:
def run_command(command, ssh=False):
    """Run a shell command locally or over SSH, capturing output and handling errors."""
    if ssh:
        command = f"ssh {remotehost} '{command}'"
    
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f"❌ Command failed: {command}")
        print("STDERR:", result.stderr.strip())
    else:
        print(f"✅ Command successful: {command}")

    return result.returncode == 0

In [ ]:
# Function to load YAML file as a dictionary
def load_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

### set your CSCS username e.g. <span style="color: blue;">cpi</span> and the CSCS account e.g. <span style="color: blue;">s1267</span>

In [ ]:

# widgets
# Create input fields
output = ipw.Output()
username_widget = ipw.Text(
    value='',
    description='CSCS Username:',
    placeholder='Enter your username'
)

account_widget = ipw.Dropdown(
    value='s1267',
    options=['s1267','s1276'],
    description='CSCS Account:',
    placeholder='Enter your CSCS account'
)


In [ ]:
def config_command(cscs_username):
    command = f"""
if grep -q 'daint.alps.cscs.ch' {config_path} 2>/dev/null; then
    echo 'No changes needed for .ssh/config: daint.alps.cscs.ch already present in {config_path}';
else
    if [ ! -f {config_path} ]; then
        cp {config_source} {config_path};
    else
        if grep -q 'ela' {config_path}; then
            cat {config_without_ela} {config_path} > {config_path}.tmp && mv {config_path}.tmp {config_path};
        else
            cat {config_source} {config_path} > {config_path}.tmp && mv {config_path}.tmp {config_path};
        fi
    fi
    sed -i 's/cscsusername/{cscs_username}/g' {config_path}
    echo 'Updated {config_path} successfully!';
fi
"""
    return command
def update_files_command(cscs_username,cscs_account):
    command = f"""
sed -i 's/cscsusername/{cscs_username}/g' {alps_files}/alps_setup.yml
sed -i 's/cscsaccount/{cscs_account}/g' {alps_files}/alps_setup.yml
sed -i 's/cscsusername/{cscs_username}/g' {alps_files}/alps_config.yml
sed -i 's/cscsusername/{cscs_username}/g' {alps_files}/cp2k.yml
"""
    return command

In [ ]:
# Define computer label and file paths
config_file = "config.yml"
setup_file = "setup.yml"
ref_config = "alps_config.yml"
ref_setup = "alps_setup.yml"

def check_install_computer():
    install_computer = True

    # Check if the computer exists
    #computer_exists_cmd = f"verdi computer list -a | grep -q '{new_host_label}'"
    process = subprocess.Popen("verdi computer list -a", shell=True, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL, text=True)
    stdout, _ = process.communicate()

    if new_host_label in stdout:
        print(f"Computer '{new_host_label}' found. Exporting configuration...")

        #subprocess.run(f"verdi computer export config {new_host_label} config.yml", shell=True, check=True)
        #subprocess.run(f"verdi computer export setup {new_host_label} setup.yml", shell=True, check=True)
        run_command(f"verdi computer export config {new_host_label} config.yml", ssh=False)
        run_command(f"verdi computer export setup {new_host_label} setup.yml", ssh=False)
        # Load and compare YAML files
        config_match = load_yaml(config_file) == load_yaml(ref_config)
        setup_match = load_yaml(setup_file) == load_yaml(ref_setup)

        if config_match and setup_match:
            print("✅ YAML files are equivalent, no need to reinstall the computer")
            install_computer = False
        else:
            print("❌ YAML files are different!")
        run_command("rm -rf config.yml setup.yml",ssh=False)
    else:
        print(f"Computer '{new_host_label}' not found. Skipping comparison.")
    return install_computer

## do first ssh connection /check ssh connection

In [ ]:
def set_ssh(cscs_username):

    command = f"ls /users/{cscs_username}"
    if run_command(command,ssh=True):
        return True

    # Loop over ela and daint hosts and add to known_hosts
    print(f"Adding {proxy} to known_hosts...")
    ssh_keyscan_command = f"ssh-keyscan -H {proxy} >> ~/.ssh/known_hosts"
    run_command(ssh_keyscan_command,ssh=False)

    print(f"Adding {remotehost} to known_hosts...")
    ssh_keyscan_command = f"ssh {proxy} ssh-keyscan -H {remotehost} >> ~/.ssh/known_hosts"
    run_command(ssh_keyscan_command,ssh=False)
    run_command(ssh_keyscan_command,ssh=False)

    # check if ssh works
    return run_command(command,ssh=True)

## relabel old daint if needed, setup new alps, and hide old codes

In [ ]:
def setup_new_alps():
    # relabel old computers
    relabel_command = f"""if verdi computer list  | grep -q '{new_host_label}'; then 
        verdi computer relabel {new_host_label} {relabeled_host} && 
        verdi computer disable {relabeled_host} aiida@localhost; 
    fi"""

    # Hide old codes that will be already @relabeled_host 
    hide_code_command = f"verdi code list | awk 'NR>2 {{print $1}}' | grep '@{relabeled_host}' | xargs -r -I {{}} verdi code hide {{}}"

    # setup ALPS
    alps_setup_command = f"verdi computer setup --config alps_setup.yml"
    alps_config_command = f"verdi computer configure core.ssh daint.alps --config alps_config.yml"
    local_commands=[relabel_command,hide_code_command,alps_setup_command,alps_config_command]
    for command in local_commands:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        # Print the output
        print(result.stdout)
        print(result.stderr)    

## setup codes

In [ ]:
def setup_codes():
    uenvs=[]
    for code_file in ['cp2k.yml', 'stm.yml', 'overlap.yml', 'pw.yml', 'pp.yml', 'projwfc.yml', 'dos.yml']:
        # Extract label from YAML file
        code_data = load_yaml(code_file)
        label = code_data.get("label")

        if not label:
            print(f"⚠️ Skipping {code_file}: No 'label' found in YAML.")
            continue
        else:
            prepend_text = code_data.get("prepend_text", "")  # Get the `prepend_text` field

            # Use regex to find the value of `--uenv=`
            match = re.search(r"#SBATCH --uenv=([\w\-/.:]+)", prepend_text)

            if match:
                uenv_value = match.group(1)  # Extract matched value
                print(f"File '{code_file}' → uenv: {uenv_value}")
                if uenv_value not in uenvs:
                    uenvs.append(uenv_value)
            else:
                print(f"⚠️ No '--uenv=' found in {code_file}")
                continue

        full_label = f"{label}@{new_host_label}"
        exported_file = "exported.yml"

        # Check if the code exists
        check_command = f"verdi code list | grep -q '{full_label}'"

        if not run_command(check_command, ssh=False):
            print(f"🔹 Code '{full_label}' not found. Installing...")
            install_command = f"verdi code create core.code.installed --config {code_file}"
            run_command(install_command, ssh=False)
        else:
            print(f"✅ Code '{full_label}' exists. Checking configuration...")

            # Export existing configuration
            export_command = f"verdi code export {full_label} {exported_file}"
            run_command(export_command, ssh=False)

            # Compare exported config with original YAML
            if load_yaml(exported_file) != code_data:
                print(f"🔄 Configuration differs for '{full_label}', relabeling and reinstalling...")

                # Relabel the existing code
                relabel_command = f"verdi code relabel {full_label} {relabeled_code}{full_label}"
                run_command(relabel_command, ssh=False)

                # Install the new code
                install_command = f"verdi code create core.code.installed --config {code_file}"
                run_command(install_command, ssh=False)
            else:
                print(f"✅ Configuration matches for '{full_label}', no changes needed.")
    run_command(f"rm -rf {exported_file}",ssh=False)
    print(f"Unevs needed: {uenvs}")
    return uenvs

## copying scripts and data directories to daint, may take a while

In [ ]:
def copy_scripts(cscs_username,remotehost):# Define commands
    ssh_commands = [
        f"mkdir -p /users/{cscs_username}/src",
    ]

    scp_commands = [
        f"scp {alps_files}/mps-wrapper.sh {remotehost}:/users/{cscs_username}/bin/",
        f"scp -r {alps_files}/cp2k {remotehost}:/users/{cscs_username}/src/"
    ]

    # Execute SSH commands
    for cmd in ssh_commands:
        if not run_command(cmd, ssh=True):
            return

    # Execute SCP commands
    for cmd in scp_commands:
        if not run_command(cmd):
            return

## Manage uenvs

In [ ]:
def manage_uenv_images(remote_host, uenvs):
    """
    Ensure that required uenv images are available on a remote host.
    
    :param remote_host: The remote machine where commands will be executed.
    :param uenvs: A list of required uenv images (e.g., ['cp2k/2024.3:v2', 'qe/7.4:v2'])
    """

    def run_ssh_command(command):
        """Runs an SSH command on the remote host and returns the output."""
        ssh_command = f"ssh {remote_host} '{command}'"
        result = subprocess.run(ssh_command, shell=True, capture_output=True, text=True)
        return result.stdout.strip()

    def extract_first_column(command_output):
        """Extracts only the first column (UENV image names) from multi-column output."""
        lines = command_output.split("\n")[1:]  # Skip the header line
        return {line.split()[0] for line in lines if line.strip()}  # Get first column values

    # Step 1: Check if the uenv repo exists, if not, create it
    print("🔍 Checking UENV repository status...")
    repo_status = run_ssh_command("uenv repo status")
    
    if "not found" in repo_status.lower() or not repo_status:
        print("⚠️ UENV repo not found. Creating repository...")
        run_ssh_command("uenv repo create")
    else:
        print("✅ UENV repo is available.")

    # Step 2: Get the list of images available to the user
    print("🔍 Fetching available UENV images (user)...")
    available_user_images = extract_first_column(run_ssh_command("uenv image ls"))

    # Step 3: Get the list of all images available on the system
    print("🔍 Fetching available UENV images (system-wide)...")
    available_host_images = extract_first_column(run_ssh_command("uenv image find"))
    available_service_images = extract_first_column(run_ssh_command("uenv image find service::"))

    # Step 4: Check missing images and pull them if necessary
    for uenv in uenvs:
        if uenv in available_user_images:
            print(f"✅ Image '{uenv}' is already available for the user.")
        elif uenv in available_host_images:
            print(f"✅ Image '{uenv}' is available on the host. Pulling...")
            run_ssh_command(f"uenv image pull {uenv}")
        elif uenv in available_service_images:
            print(f"✅ Image '{uenv}' is available in the service repo. Pulling from service::...")
            run_ssh_command(f"uenv image pull service::{uenv}")
        else:
            print(f"❌ Image '{uenv}' is not available anywhere! Manual intervention needed.")
            return

    print("✅ UENV management complete.")
    return

In [ ]:
def run_all(button):
    with output:
        output.clear_output()
        if username_widget.value == '':
            print("❌ Specify the user")
            return
        cscs_username = username_widget.value
        cscs_account = account_widget.value

        # check if we have to install teh new computer
        need_to_install = check_install_computer()
        if need_to_install:
            setup_new_alps()
        else:
            print("✅ Skipping installation of new computer and the installed computer does not need to be relabeled")
        run_command(config_command(cscs_username),ssh=False)
        run_command(update_files_command(cscs_username,cscs_account),ssh=False)
        if not set_ssh(cscs_username):
            print("❌ ssh problem, ask for support")
            return
        uenvs = setup_codes()
        if len(uenvs) >0:
            manage_uenv_images(remotehost, uenvs)
        return

In [ ]:
# Create a green "Start" button
start_button = ipw.Button(
    description="Start",
    button_style="success",  # Green color
    tooltip="Click to start",
    icon="play"  # Adds a play icon
)

# Bind the function to the button click event
start_button.on_click(run_all)

# Display the button
display(ipw.VBox([username_widget,account_widget,start_button,output]))